---
>「不用意にもらす言葉こそ、ほんとうらしいものをふくんでいるのだ。 」
>
> 太宰治
---

# はじめに

ここでは、LangChainを主として、ChatGPTなどのLLMモデルを便利に使うための仕組みについて学ぶ

なお、ChatGPTを対象とする場合、ChatGPTには、GPTsという独自のChatGPTをNoCodeで作成するツールが準備されている
- 従って、以下で説明する内容の多くは、この機能を用いて実現できる
- ただし、仕様がすぐに変更されるなど、混乱している状況にあるため、各自で触って試してみるとよい
- GPTsの開発状況により状況が大きく変わる可能性がある点に注意する事

# LangChain
LangChainは日々更新されている
- これは、この授業テキスト全般に言えることであるが、特に後半は更新が頻繁に行われている
- アップデートにより実行できない場合もあるが、その場合は速やかに申し出ること

その前に、OpenAPIのChat APIについて学ぶ

なお、このノートブックは、GPUを使わないCPUランタイムを利用している

***注意***

ChatGPTは大人気のサービスのため負荷が集中しており、特にAPIの利用が多いため、無償料金枠が2024年時点でなくなっている

利用においてもし、次のような文章を含むエラーが出力された場合は、しばらく待って再度実行する必要がある

```
WARNING:RateLimitError: Rate limit reached for default-gpt-3.5-turbo on requests per min. Limit: 3 / min. Please try again in 20s.
```

もし、まとめて実行する場合、途中で実行を待ってスロットを使いつくさないようにする必要があるため、次の設定を行うとよい
- ***制約がかかった場合は、openai_wait = Trueにすること***
- 利用にはまずAPIキーを発行してください

In [ ]:
import time
#openai_wait = True
openai_wait = False

また、途中でバージョン不一致によるエラーを回避するため、次のコードを実行後、速やかにセッションを再帰同すること(2024年時点で問題発生)

In [ ]:
!pip install --upgrade nltk
import nltk

# Chat APIを利用する


## API Keyの発行

openai.comにアクセスし、DASHBOARDメニューにあるAPI keysでAPI keyを発行する
- セキュリティのため、プロジェクトごとに異なるキーを利用すること
- ここでは、dataai-keyという鍵をつくるとよい
- 発行された鍵をコピーしておくこと

## API Keyを使えるようにする

発行したKeyを次のコードにペーストして利用する

次のセルにある、```%env OPENAI_API_KEY=``` に続けて、APIキーを記録して実行すること

In [ ]:
%env OPENAI_API_KEY=ここにキーを登録

env: OPENAI_API_KEY=ここにキーを登録


In [ ]:
import os
print(os.environ['OPENAI_API_KEY'])

ここにキーを登録


APIはOpenAI Chat APIに統一された

モデルは、コストの安いgpt-3.5-turboを利用する
- ここでは性能よりも使い方を学ぶためコスト重視とする
- model="gpt-4"などとすることで、GPT-4を利用することができるが、利用料が高くなることに注意すること

## APIを使う

まず、ChatGPTに挨拶して、APIが使えているかどうかを確認する

"Hello. Am I using the API correctly?"

と聞いて回答を実際に得る

必要なライブラリは、単純に次の2つ
- インターネットを利用して、シンプルにRESTでリクエストを投げてレスポンスを得るためのrequests
- JSONフォーマットを扱うためのjson

In [ ]:
import requests
import os
import json

In [ ]:
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role": "user", "content": "Hello. Am I using the API correctly?"}
    ],
    "temperature": 0,
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

{
  "id": "chatcmpl-Ao0REWPrxzGIxQKDXArFYM1NCiJyn",
  "object": "chat.completion",
  "created": 1736481448,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! I'm an AI assistant and I'm here to help you with any questions you may have about using APIs. Can you provide more details about the API you are using and what specific issues you are facing?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 44,
    "total_tokens": 60,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


返事は"content"にあるように、

"Hello! I'm an AI language model and I'm here to help you. Could you please provide more details about the API you are using and what you are trying to achieve?"

となるが、意味としては、

「こんにちは！私はAI言語モデルです。使用されているAPIの詳細と、何を達成しようとしているのかを教えていただけますか？」

となる

なお、パラメータについて、
- temperatureは0に近いほど、同じ回答を出力するようになる
- max_tokensは返答として最大何文字返すかを指定する
  - ChatGPTのAPIは、利用文字数(トークン数)により課金されるため、コスト低減を考えるのであれば重要である  
  デフォルトは16であるが、これは通常の利用ではかなり少ないといえる
- nは同一質問に対する返答数を指定する  
  この場合temparatureを大きめの値にしなければ、同じ回答が並ぶことになる

これで、ひとまず使えるようになったであろう

### エラートラブル(1)

次のように表示される場合は、APIを利用するための無償枠を既に使い切ったか、時間が過ぎたためExpireしたことを意味する

```
{
  "error": {
    "message": "You exceeded your current quota, please check your plan and billing details.",
    "type": "insufficient_quota",
    "param": null,
    "code": "insufficient_quota"
  }
}
```

- 無償で続けたい場合は、新しいアカウントをつくるとよい
- もちろん、コストを支払ってもよい

新しいアカウントをつくる

- ただし、メールアドレスが必要となる
  - 既にメールアドレスが枯渇している場合は、フリーメールアドレスを取得するとよい

- ブラウザのシークレットモードで、openai.comを開く(クッキーで既に持っているアカウント情報を利用しないようにするため)

- 右上のメニューからLoginを選択し、Sign upを選択する
  - メールアドレスとパスワードを入力する
  - 確認メールが届くのでVerifyする

- OpenAIのページに行くと、名前や誕生日の入力が求められる

- スマートフォンの番号を入れて、コードを受け取る
  - 現状では利用済の番号でも問題ないが、将来は不明である

- APIをクリックして、OpenAI platformに行き、右上の丸いアイコンをクリックして個人メニューに入る

- 左のタブでAPI Keysを選択する

- "Create new secret key"を選択し、dataai-keyという鍵を作成してコピーしておく

以上、あらたに入手した鍵を利用して再実行すること
- ただし5ドル分しかないので注意すること

### エラートラブル(2)

頻繁にアクセスすると、エラーが発生する

これは、単位時間あたりのアクセス数が制限されているためである
- しばらく待ってリトライすること

### roleについて

"role"は、次の3つがある
- userはChatGPTのユーザで、皆さんのこと
- assistantがChatGPTによる回答を指し、ChatGPTのこと
- systemはassistantのふるまいを制御するために利用

但し、gpt-3.5-turboでは、systemは利用しない


## API利用において重要なこと

### ブラウザ版との違い

API利用と、ブラウザ利用における最も大きな違いは、API利用では過去の会話のやり取りを一切考慮しないという点である

したがって、APIを利用する場合で過去の会話を参照したい場合は、過去の会話そのものを全てmessageに記載する必要がある


### 料金の確認

https://openai.com/pricing にアクセスすると、

| Model	| Input	| Output |
|:---|:---|:---|
| GPT-3.5-Turbo 4K context | \$0.0015 / 1K tokens | \$0.002 / 1K tokens |

と記載されている

現在いくらつかったかは、

https://platform.openai.com/account/usage

にアクセスして確認するとよい


### トークン数

課金対象にもなっているトークン数について、トークンは基本的に単語のことであり、トークン数は入力した単語の数を意味する

ChatGPTを含む多くのLMにおいて、膨大な単語を効率よく学習するため、一つの単語を複数のトークンに分割して処理している
- 例えば、"humburger"は、"hum", "bur", "ger"の3つに分解される

また、日本語と英語ではトークン数のカウント方法が異なる
- 日本語は内部で英語に変換されて処理されているため日本語は似た内容の文章において課金上不利となる

- 日本語は1トークンはおよそ4文字、英語ではおよそ0.75語に相当する
  - 一般に直接英語を利用した方がお得といわれている所以
  - 実際2倍程度の開きがある

このトークン数は、各モデルの入力や出力サイズの制限にも利用される

直接文字数を計数したい場合は、https://platform.openai.com/tokenizer にアクセスして、文章を入力するとよい




# GPT APIを用いたアプリケーション実装

***ここでは、料理名を入力することで、材料と手順、調理時間、お勧めのサイドメニューなどを表示するというアプリを作成することを念頭に説明する***

## 全体の構成

### APIまでの通信手順

以下の手順を踏む
- スマートフォンやPC、Webのアプリを利用して、レシピ名をサーバプログラムに送信する
- サーバプログラムは、プロンプトエンジニアリングを行い、APIキーを付与してOpenAIのAPIを叩く

レシピ生成アプリが直接OpenAI APIを叩くような構成は、APIキーが漏えいするため普通は行わない

<img src="http://class.west.sd.keio.ac.jp/dataai/text/chatapi1.jpg" width=700>

# LangChain

## LangChainとは？

LLMを使ったアプリケーション開発フレームワーク
- PythonとJavaScript/TypeScriptの2つがある
- フリーで利用できる

詳細は公式のドキュメントを参照すること
- 過去のバージョンのマニュアルを見る場合は、githubにあるlangchainに行き、docsのreleasesから目的のバージョンを選択するとよい

## Module

LangChainにはmoduleと呼ばれる構成要素がある
- moduleとして、Models, Prompts, Chains, Indexes, Memory, Agentsがある

### Models module

LongChainで利用する機械学習モデルである

- Chat Models: Open AIのChatAPIのためのモジュール
- Text Embedding Models: テキストをベクトル化するモデル



先に示した簡単な対話プログラムを再構成する
- 先ほどよりもシンプルに記述できることがわかるであろう
- 内部でjsonに変換され通信が行なわれている

openaiをインストールする

ライブラリの開発速度が速く、バージョン競合が簡単に発生するため注意すること
- できれば、自分で解決する能力を身に着けるとよい
- (2023/12) openai、cohere、tiktokenは同時に導入しないと警告が表示される
- (2023/12) tensorflow-probabilityと一部ライブラリがコンフリクトするため、uninstallする

次のようにインストールすることで対処する

In [ ]:
!pip uninstall -y --quiet tensorflow-probability tensorflow-metadata tensorflow-datasets

In [ ]:
!pip install --quiet openai cohere tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.2/250.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.5 MB/s eta 0:00:00


In [ ]:
!pip install --quiet chromadb kaleido python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install --quiet langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00


次のようにpredict関数を用いることで、問い合わせと応答を行うことができる

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = llm.invoke("自己紹介してください。")
print(result.content)

### Prompts module

モデルへの入力を組み立てるmoduleであり、次の要素がある
- Prompt Templates
- Chat Prompt Templates
- Example Selectors
- Output Parsers

ここでは、Prompt Templatesについて説明する
- ChatGPTへのプロンプトについてテンプレートつまり例文を作成することができる



次のコードでは、commandがlsに置き換わる
- Promptの長さを考慮して埋め込む
- 出力する形式を指定して埋め込む

などが可能であり、単純なPythonコードによる埋め込みよりも高度な処理が可能である

In [ ]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

result = prompt.format(command="ls")
print(result)

### Chains module

Models, Templates, Chainsなどのmoduleを連結する

なお、LangChainの挙動の詳細を確認するため、
`langchain.verbose = True`
としている
- いろいろと意味のない文章や宣伝も表示されるが、不要な場合は、Falseとするとよい

In [ ]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Prompt を用意
template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""
prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

# Chain を作成
chain = LLMChain(llm=chat, prompt=prompt)

# 実行
result = chain.invoke("ls")
print(result)

chain.invokeを用いて、構築したchainが順に実行される
- 最初に文字の埋め込み(prompt)が行なわれる
- 次にchat modelにより実際に通信が行なわれる

このChainには各種存在する

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

#### SimpleSequentialChain

ChainとChainを直列に連結する



例えば、次の例を考えてみよう
- 簡単な算数の問題を問い合わせ、それに対して回答を得る場合、詳細な手順を聞くようにすると解答の精度が向上する
- しかしながら、欲しいのは最終的な答えであって、途中経過は不要であるとする
- すると、まず、詳細な手順を含む答えを得てから、その答えを要約して最後の答えだけ得るようにするとよい

つまり、ChatGPTを2回利用して最終的に欲しい回答を獲得することになる

In [ ]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 1 つ目の Prompt と Chain を用意
cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template,
)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

# 2 つ目の Prompt と Chain を用意
summarize_template = """
入力を結論だけ抜き出して記述してください。

### 入力 ###
{input}
### 入力終了 ###
"""
summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template,
)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

# 2 つの Chain を直列に繋ぐ
cot_summarize_chain = SimpleSequentialChain(
    chains=[cot_chain, summarize_chain])

# 実行
result = cot_summarize_chain(
    "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？")
print(result["output"])

なお、独自のpromptsやchains moduleを作成することも可能である

詳細は、LangChainのマニュアル https://python.langchain.com/docs/get_started/introduction を参照されたい


#### Output Parsers

出力形式を指定するプロンプトの作成とPythonオブジェクトとのマッピングを提供する

<img src="http://class.west.sd.keio.ac.jp/dataai/text/chatapi2.jpg" width=700>

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")
    time: List[str] = Field(description="time to make the dish")
    sides: List[str] = Field(description="side menu of the dish")
#    tools: List[str] = Field(description="tools which are required to cook the dish")

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=chat, prompt=prompt)

output = chain.run(dish="カレー")
#output = chain.run(dish="インド本格カレー")
print("=== output ===")
print(output)

最後に、recipe型にマッピングする

In [ ]:
recipe = parser.parse(output)
print("=== recipe object ===")
print(recipe)

In [ ]:
recipe.ingredients

### Indexes

ChatGPTは学習に用いたデータセットの範疇でのみ答えを出すため、新しい知識や概念については、正しく解答することが難しい

例えば、次のような質問に対しては、お手上げになっている

<img src="http://class.west.sd.keio.ac.jp/dataai/text/langchain1.jpg" width=500>

ここで、質問に対する回答を得るうえで必要となる情報を渡してから処理させると、おおよそ正しい回答を得ることができるようになる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/langchain2.jpg" width=500>


コンテキストとして、情報を加えることで正しい回答を与えることができており、これは強力な方法であるが、実際に用いる場合は注意が必要である

- ChatGPTには、入力文字数に制限があるため、長い文章を入力する必要がある場合はともかく、様々な情報を大量に与えておいて、そこから適切な回答を得るという利用は困難である
- 文字数、つまり入力トークン数も課金に関係するため、高コストとなる

まず、全ての情報を与えておいて、何かしら回答を得るということは非現実的であるが、自動化という点では有効な手段である



#### Vector Store

そこで、Vector Storeを活用する
- 文章をベクトル化してVector Storeに保存、入力と近しいベクトルの文章をVector Storeから検索してcontextに含める手法
- 全文章ではなく、「大事と思われる部分文章群についてのみ」情報を与える
  - これには、、embeddingと呼ばれる内部ベクトル表現への変換を行い、そのベクトルの近接性を用いて、どの文章が重要かを判断している
  - 単語をベクトル化するトークンではない点に注意すること

<img src="http://class.west.sd.keio.ac.jp/dataai/text/langchain3.jpg" width=700>



実際にindexを扱う

ここでは、LangChainのドキュメントを参考にして、質問できるようにする

最新のドキュメントはgitで公開されているため、LangChainのgitをcloneする

In [ ]:
!git clone https://github.com/hwchase17/langchain.git

Cloning into 'langchain'...
remote: Enumerating objects: 224717, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (325/325), done.


langchainのディレクトリに入る

In [ ]:
!cd langchain

以降の動作確認で必要となるライブラリを導入する

In [ ]:
!pip install --quiet unstructured tabulate pdf2image pytesseract chromadb tiktoken

まず、ディレクトリの中の全体を読むための便利なDirectoryLoaderを用いて、ある場所にあるファイルをサブディレクトリもまとめて取得する
- ここでは拡張子がmdであるファイルに限定している

- そこから次々に文章を取得して、VectorstoreIndexCreatorで、Vectorsoreに格納していく
  - この時、embeddingと呼ばれる内部ベクトル表現への変換も同時に行っている

In [ ]:
!pip install --quiet nltk click joblib regex tqdm

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
nltk.download('punkt')
loader = DirectoryLoader("./langchain/docs/", glob="**/*.mdx")
embeddings = OpenAIEmbeddings()
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

では、そのvector storeを利用して、質問する

なお、現時点でChatGPTに"LangChain"について問い合わせると、データセットに含まれていないという回答になる

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = index.query("LangChainにおけるIndex moduleについて概要を1文で説明してください。", llm=chat)
print(result)

`langchain.verbose = True`のため、無意味な動作確認メッセージも表示されているが、要約文が出力されていることがわかる
- なお、若干回答は的を得ていない


DocumentLoadersは、webサイト、GoogleDrive、Slackなどを読み込む機能が存在しているため、様々な情報をVctor Storeに格納することができる

このように、ある特定分野のデータを一連の要素としてエンコードし、それぞれが内部で 1 つの「ベクトル」として表現している
- これには、単語であればWord2Vecなどが利用できるが、ここでは文章であることから、BERTをFine-TuningしたSentenceTransformerの利用が検討される
- このベクトル数値は、多次元ベクトル空間で要素を相互に関連づけてマッピングしている

ベクトル要素がセマンティックであり、ある一つの意味を表していると考えるならば、そのベクトルの近接性が文脈関係の指標となりえる
- このベクトルはエンベディングと呼ばれる
- 互いに関連性のある意味要素がまとまって配置されるようにエンベディングを行う

特定分野の文脈によって、セマンティック要素は単語、フレーズ、センテンス、パラグラフ、文書全体、画像、あるいはまったく別のものになる可能性があり、エンベディングが最善というわけではない



プロンプトで必要となる文脈を生成するため、データベースに問い合わせを行い、ベクトル空間の入力と密接に関連する要素を抽出する必要がある

ベクトルデータストアは、大量のベクトルを保存し、問い合わせに答えるシステム
- 効率的な最近傍クエリアルゴリズム(k-NNなど)と適切なインデックスにより、データ検索を行う

### Memory

ChatGPTをブラウザで利用した場合、過去の会話の履歴を踏まえて返答するが、APIではそのような振る舞いは行わない

APIを利用する場合は、過去の履歴をプロンプトに入力する必要がある

実際にその振る舞いを確認する

次のような関数を用意してAPIを用いて文章を渡す

In [ ]:
def post_chat_completions(content):
  url = "https://api.openai.com/v1/chat/completions"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
  }
  data = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "user", "content": content}
      ],
      "temperature": 0,
  }

  response = requests.post(url=url, headers=headers, json=data)
  print(json.dumps(response.json(), indent=2))

では、実際に名前を伝える

In [ ]:
post_chat_completions("Hi! I'm Keio Yukichi!")

"Hello Keio Yukichi! How can I assist you today?" といった回答が得られているであろう
- Generativeであるため、この答えは毎回異なる

その上で、名前を憶えているか聞いてみよう

In [ ]:
post_chat_completions("Do you know my name?")

当然であるが、知らないという答えになる

そこで、過去の会話の履歴を全て入れて、同じ質問を行ってみよう

In [ ]:
post_chat_completions("""A: Hi! I'm Keio Yukichi!
B: Hello Keio Yukichi! How can I assist you today?
A: Do you know my name?
B: """)

となり、今度は"Yes, you introduced yourself as Keio Yukichi."と回答している

A:やB:は、会話しているのがどちらかを示す識別子であり、どのような形でもよい
- ただしLangChainは、内部でhumanとAIという用語を利用している

このように、会話の過去の履歴を含めて問い合わせを行うため、過去の履歴を記録し、挿入するという処理が必要となる
- これがMemory moduleの役割である

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

実際に使うには、Chainの中にmemoryとしてConversationBufferMemory()を加えるだけでよい

プロンプト(文字を入力するための入力窓)が出てきたら、
- Hello. I'm Keio Yukichi
- Do you know my name?
- EOC
と入力する

EOCは会話を終了させるおまじないである
- これは、ChatGPTの機能ではなく、そのようにプログラムしている

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    if(user_message == 'EOC'):
      break
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

<ipython-input-30-5dafd31b1180>:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
<ipython-input-30-5dafd31b1180>:9: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


roleには、system, assistant, userの3つがあることは述べた
- systemは設定に利用され、例えばChatGPTにキャラを与えるような命令も存在する

assistantとuserについて、assistant はAIからの回答、 user はユーザーからの発話であり、基本的にuserとして聞きたいことや、会話履歴を含めてを送ることになる
- この例では、userに、ユーザとChatGPTの両方の会話を入れ込んでいる

しかしながら、本来は、assistantがAIからの回答であることから、userにはユーザからの発話履歴のみ、assistantはAIからの発話履歴のみを入れるという形が望ましい
- これについては後で説明する

その他、様々なMemory moduleが提供されている
- ConversationBufferWindowMemory
  - ある範囲の会話履歴のみ入力する
- ConversationSummaryMemory
  - 会話履歴の要約を入力する

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

### Agents

LLMが必要に応じて様々なタスクを実行すると便利と思うであろう

例えば、
- 検索エンジンで検索させる
- 実際にコマンドを実行させる
- プログラミング言語やスクリプト言語でコードを実行させる

これを行うのがAgents moduleである

Agentsの利用により、実際にはLLMが何かを操作するわけではないが、LLMが何かしらアプリを操作しているかのように動作させることができる

Agentsで操作可能なアプリの例
- bash (シェル)
- Google Search
- IFTTT WebHooks (スマートホーム等)
- Python REPL
- Requests (他のAPIを叩く)
- Wikipedia API

実際にAgentsを利用してみよう


In [ ]:
!pip install --quiet langchain-experimental

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], llm=chat, allow_dangerous_tools=True)
agent_chain = initialize_agent(
    tools, chat, agent="zero-shot-react-description")

result = agent_chain.run("What is your current directory?")
print(result)

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

単純に/content というディレクトリにいるという回答であるが、実際正解である

ただ、ここで疑問が生じる **なぜ、ChatGPTはこちらのシェル環境のカレントディレクトリがわかったのだろうか？**

これは、AgentsがMRKL(ミラクル)やReActなどを利用して動作しているためである
- MRKL(Multi-Round Knowledge Loop)
- ReAct (Reasoning/Acting) なお、Reactではない


ログをみると、LangChainは次のようなプロンプトを生成している
```
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
```

翻訳すると、
```
次の質問にできるだけ答えてください。あなたは以下のツールにアクセスできる：

terminal: このLinuxマシンでシェルコマンドを実行する。

以下の書式を使う：

Question: あなたが答えなければならない入力問題
Thought: 何をすべきかを常に考える。
Action: 取るべき行動。[terminal]のどれかであるべき。
Action Input: アクションへの入力
Observation: 行動の結果
...（このThought/Action/Action Input/ObservationはN回繰り返すことができる）
Thought: 最終的な答えがわかった
Final Answer: 元の入力された質問に対する最終的な答え

始める！
```

となっており、これらの書式を用いて処理が進む
```
Question: What is your current directory?
```
という問いかけに対して、
```
Thought:I can use the "pwd" command to find out the current directory.
Action: terminal
Action Input: pwd
```
とChatGPTが返答する

そこで、AgentはAction Inputに記載されているコマンドを実行する
- その結果を Observationとして埋め込む

さらに質問を続けるが、先のプロンプトに加えて、次の文章が加わっている
- つまり、これまでの動作をプロンプトに入力している

```
Question: What is your current directory?
Thought:I can use the "pwd" command to find out the current directory.
Action: terminal
Action Input: pwd
Observation: /content

Thought:
```
これに対して
```
I now know the final answer
Final Answer: The current directory is /content.
```
と回答している

Final Answerとして現在のディレクトリは/contentであることが示されており、Final Answerが返されたので、実行を終了している

では、どんどんやってみよう

In [ ]:
result = agent_chain.run("Make a new directory called 'testdir-by-agent'")
print(result)

ファイルも作ってみよう

なお、途中で無料枠の場合はスロットを使い切ってしまうので、ワーニングメッセージと待ちが発生する

In [ ]:
result = agent_chain.run("Create new file called test.txt in the directory of testdir-by-agent and store the text of This is test in the file.")
print(result)

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

testdir-by-agentの下にtest.txtがあり、その中身がThis is a testであることを確認しよう

### MRKL(Multi-Round Knowledge Loop)

例えば、「現在の日本の首相の年齢から現在のフランスの首相の年齢を引いたらいくつですか？」という問いに対して、ChatGPTは答えることができるか？

これを直接WebのChatGPTに問い合わせても答えることができない

しかしながら、APIでは答えることができる
- つまり、APIでChatGPTをアクセスすると、Webとは異なる仕組みでアクセスできるということ

では、その手順であるが、まず質問に答えるための手順を考える

- 最初に問い。「現在の日本の首相の年齢から現在のフランスの首相の年齢を引いたらいくつですか？」（Question）

- Googleで未知の情報を調べるために検索ワードを考える(Thought)

- 現在の日本とフランスの首相の年齢を知る必要がある
  - 検索ワードは「現在の日本の首相の年齢」(Action Input)
  - さらに検索ワードは「現在のフランス首相の年齢」(Action Input)

- 検索を実行「現在の日本の首相の年齢」（Action）
  - 結果は65歳でした（obsabation)

- 検索を実行します「現在のフランス首相の年齢」（Action）
  - 結果は61歳でした（obsabation)

- 日本の首相の年齢とフランスの首相の年齢の差分を計算する必要がある（Thought）

- 計算（Action）

- 結果は4でした(obsabation)

- 答えは4歳です (final)

このようにMRKLは、ChatGPTが情報をもとに次のアクションを考え、結果を評価し、次のアクションを考えるというプロセスを繰り返すことで回答精度を上げる方法論である

考察（Thought）、観察（Observation）、行動（Action）のサイクルを繰り返すことで、回答精度が向上する

よく言われる、ステップバイステップで考えるように指示すると正答率が上がるのと似ているが、Agent側で実際に実行して応答できるように工夫されている

### Prompt Coding

プロンプトコーディングはChatGPTの活用において必須となる技術である

ChatGPTから精度の高い回答を得るために、人間に質問するのと同様に、質問力が重要であり、その質問の仕方に関する研究が進められている

例えば、以下のように役割の指定や回答の形式を細かく設定することで、正答率を上げることができる
- 質問や回答が定型化されており、プログラムで文字列を処理することが容易になり、解析が可能となる

```
あなたは、英語の先生です。これから私の英語を英語教師として文法の誤りを訂正して下ださい。
回答のフォーマットは以下のようにします。
あなたの英語：{入力分}
訂正後の英文:{英文例}
文法の解説:{解説1000文字以内}
```

### 実際のプロンプト

先の年齢差を問う問題に答えさせる場合、次のようなプロンプトが想定される
- 内容は、シェルを実行するAgentの問い合わせと酷似する

```
Answer the following questions as best you can.
You have access to the following tools:\n\n

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\n
Calculator: Useful for when you need to answer questions about math.\n\n

Use the following format:\n\n

Question: the input question you must answer\n
Thought: you should always think about what to do\n
Action: the action to take, should be one of [Search, Calculator]\n
Action Input: the input to the action\n
Observation: the result of the action\n
... (this Thought/Action/Action Input/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question\n\n

Begin!\n\n


Question: 現在の日本の首相の年齢から現在のフランスの首相の年齢を引いたらいくつですか？ 計算してください\nThought:')
```

最初の"Use the following format:\n\n" 以前について、

ここで、toolsについて何がどのように利用できるかを伝えているが、重要な点は次の通りである

- Searchというツール名
  - コロンの前にツール名が記載されている
- ユースケースを伝える
  - (時事問題に関する質問に答える必要があるときに便利です)
- 入力形式を指定する
  - (入力は、検索クエリである必要がある)

「入力は検索クエリである必要がある」と伝えているため、半角スペース区切りの単語単位での検索クエリを作成するようになる
- ChatGPTは時事問題に関する内容は、Searchツールを使うようになる

最初の"Use the following format:\n\n" 以降について

進め方とフォーマットを伝えている

- Question:質問内容を記載
- Thought:何をすべきかを常に考える必要がある
  - アクションを考えるように指示
- Action: 実行するアクションは、 [Search, Calculator]のいずれかである必要がある
  - アクション名はツール名と同じであり、ChatGPTからActionの指示が出る際には[Search,Calculator]のキーワードが出力される
- Action Input: アクションへの入力
  - Searchの場合は指示されたクエリ形式で入力する
- Observation:Actionの結果
  - アクションの結果を表示

さらに、最後について

- (this Thought/Action/Action Input/Observation can repeat N times)
  - N回繰り返すは、答えが出ない場合に打ち切る回数や、API利用料金を抑えるための制限回数として、Nを指定できるようにしている
- Thought:回答が判明したら下記に進みます
- Final Answer:最終的な回答をします

実際に試行すると次のような結果を得ることができる

```
> Entering new AgentExecutor chain...
I need to find out the age of the current Japanese and French Prime Ministers
Action: Search
Action Input: "age of current Japanese Prime Minister"params

Observation: 65歳
Thought: Now I need to find out the age of the current French Prime Minister
Action: Search
Action Input: "age of current French Prime Minister"params

Observation: 61歳
Thought: I now know the final answer
Final Answer: 4歳
```



##  Chat API におけるプロンプトの構築

先のmemoryの例に加えて、
`import openai`

および

`langchain.verbose = True`

を追加して、ログを詳細に取得する

プロンプトに対して、
- Hi, I'm Keio Yukichi.
- Do you know my name?
- EOC

と入力する


In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import openai

langchain.verbose = True
openai.log = "debug"

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    if(user_message == 'EOC'):
      break
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

ログを参照することで、動作の詳細を獲得できる

例えば、Memoryにより過去の履歴をプロンプトを与えることができるが、具体的には次のような動作をしている

```
api_version=None data='{"messages": [{"role": "user", "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\\n\\nCurrent conversation:\\nHuman: Hi, I\'m Keio Yukichi.\\nAI: Hello Keio Yukichi! How can I assist you today?\\nHuman: Do you know my name?\\nAI:"}], "model": "gpt-3.5-turbo", "max_tokens": 100, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'
```

このように、すべてuserメッセージとして混入している

本来は、ChatGPTの言葉は、assistantとして入力するべきであろうことがわかる

そこで、これを使い分けるには、次のようにする

SystemMessage, HumanMessage、またAIMessageを用いて、それぞれの会話を仕分けできる

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi! I'm Keio Yukichi!"),
    AIMessage(content="Yes, You are Keio Yukichi.")
]

result = chat(messages)
print(result)

In [ ]:
#スロット待ち
if openai_wait:
  time.sleep(60)

これを踏まえて、先ほどのループ問い合わせプログラムを改善する

```
    memory.chat_memory.add_user_message(user_message)
    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
```
とすることで、memoryに対してだれの発言かを仕分けして登録するようにする

実際に実行して、次のようにプロンプトに入力する
- Hi. I'm Keio Yukichi.
- Do you know my name?
- EOC

ログを見てみると、Do you know my name? の問い合わせの後、

`"messages": [{"role": "user", "content": "Hi. I\'m Keio Yukichi."}`とuserが入力した後、`{"role": "assistant", "content": "Hello Keio Yukichi! How can I assist you today?"}'とassistantが返答、さらに`{"role": "user", "content": "Do you know my name?"}`とuserが入力といった具合に、正しく仕分けされている


In [ ]:
langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory()

while True:
    user_message = input("You: ")
    if(user_message == 'EOC'):
      break
    memory.chat_memory.add_user_message(user_message)
    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
    print(f"AI: {ai_message.content}")


ここで一度実行を終了する
- 以降は「ランタイム」から「以降のセルを実行する」を選択して実行するとよい
- もしくは一つ一つクリックして実行すること

In [ ]:
from google.colab import runtime
runtime.unassign()

# Embedchain



Embedchainは、どのようなデータセットでも簡単にLLMボットを作成できるフレームワークである

- CSVやExcel、テキストファイルに加えて、YouTube動画など、様々なフォーマットを扱うことができる
- バックエンドで、ChatGPT API、OpenAIのembeddingとLangChain、ベクトルデータベースにChromaを利用している



## ライブラリのインストール

次のセルでlangchainをインストールするが、dependancyでエラーが生成されるかもしれない
- (2023/12) embedchainの都合により、バージョンを指定して導入する

In [ ]:
!pip install --quiet langchain==0.0.336

In [ ]:
!pip uninstall -y --quiet tensorflow-probability

In [ ]:
!pip install --quiet openai cohere tiktoken

In [ ]:
!pip install --quiet chromadb kaleido python-multipart

In [ ]:
!pip install --quiet --upgrade embedchain

In [ ]:
!pip install --quiet pydantic

必要なライブラリをインポートする

In [ ]:
import os
from embedchain import App

APIキーは、先に指定したキーを利用する点に注意する

次のようにして、直接キーを指定してもよい

```
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

```

In [ ]:
!pip install python-dotenv

次のコードでTrueと出ればよいが、Falseと表示された場合は、戻って最初の方にある"!echo ..."のセルをもう一度実行するとよい

In [ ]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

embechainボットを起動する
- ここでは、ChatGPTを起動している
- Llama2App()とすると、Llama2が起動する
- その他、CustomAppなどを用いることで様々なLLMモデルに対応する

In [ ]:
elon_bot = App()

次に、embedchainの `.add()` メソッドを使って異なる種類のデータソースを追加する

- `"pdf_file"`でPDFファイルをURLなどで追加できる
- その他の対応フォーマットについては、https://docs.embedchain.ai/advanced/data_types を参照のこと


In [ ]:
!pip install --quiet youtube-transcript-api
!pip install --quiet pytube

In [ ]:
elon_bot.add("web_page", "https://en.wikipedia.org/wiki/Elon_Musk")
elon_bot.add("youtube_video", "https://www.youtube.com/watch?v=kwsTrVoCfSQ")

ボットの準備ができたので、`.query()`メソッドを使ってボットに質問する

なお、ここでは、queryを用いているが、2つのインタフェースがある

- クエリー・インターフェース
  - このインターフェイスは質問に答えるボットであり、質問を受け、その答えを取得する
  - 過去のチャットに関するコンテキストは保持しない
  - `.query()`関数を呼び出して、任意のクエリの答えを取得できる

- チャット・インターフェース
  - 過去の会話を記憶するチャット・インターフェイスであり、デフォルトでは過去5つの会話を記憶している
  - `.chat`関数を呼び出して問い合わせの答えを取得できる

- ドライ・ラン
  - 追加、クエリー、チャットの各メソッドにあるオプションで、LLMに送信せず、生成されたプロンプトを表示することができまる


```
chatmsg = naval_chat_bot.query('Can you xxxxxx?', dry_run=True)
```

- ストリーム応答
ChatGPT のようにレスポンスをストリームするために、クエリメソッドに追加するオプション
- チャンクを希望のフォーマットでレンダリングするには、ダウンストリームハンドラが必要となる
- OpenAIモデルとOpenSourceAppの両方をサポートする

```
app = App()
query_config = QueryConfig(stream = True)
resp = app.query("What is ****** ?", query_config)

```

- リセット
  - `reset`でデータベースをリセットし、すべての埋め込みを不可逆に削除する

- カウント
  - `count`でデータベース内の埋め込み（チャンク）の数を数えます。

In [ ]:
elon_bot.query("How many companies does Elon Musk run?")

# Gradioについて

***ここからは、まとめて実行せず、つ一つ一つクリックして実行した方が良い***


Webアプリを簡単に実装できるPythonライブラリ

百聞は一見に如かずということで、早速実行してみよう

## gradio のインストール方法

```
pip install gradio
```

とし、例えば、

```
import gradio as gr
```
とすることで利用可能となる
- (2023/12) ここでは互換性のために3.48.0を導入する

In [ ]:
!pip -q install gradio==3.48.0

シンプルなWeb UIを作成して起動させる例を示す
- 名前を入力し、名前へのあいさつを出力するWeb UIである

Colab上で実行すると、Colabのwebに統合される
- URLが出力されている通り、そのURLにアクセスできる環境にあれば、webのページとして表示される
- この例では、`https://localhost:7860/`などと表示されているが、クリックすると実は`https://wq0lbccui9-496ff2e9c6d22116-7860-colab.googleusercontent.com/`に転送されており、ネットワークセキュリティ上隔離されたColabの外からアクセスできるようになる
- Colab環境では、セキュリティ上の問題もあり、このようなグローバルアドレスが提供されない場合は、別途ボートフォーワーディングなどの知識が必要な場合がある
- また、ローカル上で他のWeb UIアプリを動作させているなどにより、ポートが競合する可能性がある
  - この場合、7860 が 7861 や 7862 といった番号に変わることがある


また、gradio clientを用いることで、作成したwebアプリケーションをweb APIのように利用することもできる
- 下に小さく「Use via API」と記載されているが、これをクリックし、記載の通りに実行すると動作がわかるであろう
·

In [ ]:
import gradio as gr

# あいさつの関数
def greet(name):
    return "Hello " + name + "!"

# Interfaceの作成
demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs="text"
)

# 起動
demo.launch()

## 設計手順

次の手順となる
- コールバック関数を定義
- レイアウトを定義
- WebUIの起動

それぞれについて概要をみてみよう


### コールバック関数

まず、コールバック関数として、画面レイアウトでボタンが押された時に呼び出したい関数を記述する  

例えば、名前(text)、表示フラグ(boolean)、値(0から100の値)の3つを受け取る関数として次のような関数を想定する  
```
def my_func(my_name, is_disp, my_value):
    return f'### {my_name} ###', my_value * 100
```



### Interface

「Interface」は、関数をUIでラップするためのクラスであり、主なパラメータは次のとおり
- fn : ラップする関数
- inputs : 入力コンポーネント ("text"、"image"、"audio"など)
- outputs : 出力コンポーネント ("text"、"image"、"label"など)

画面レイアウトの作成として、画面に表示したいUIパーツと、ボタンなどが押された時のアクション(呼ぶ出したいコールバック関数の名前)を記述する  
- 入力UIとして、my_nameは"text"であり、is_dispは"checkbox"であり、my_valueは、0から100の値をスライダーで入力させるとすると、gr.Slider(0, 100)となり、上からこの順に入力UIがレイアウトされる
- 出力UIとして、ここでは、文字と数字を想定する

最終的に、次のような関数定義となる  
```
demo = gr.Interface(
    fn = my_func,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)
```

なお、"text"に対して、より詳細な情報を与えることもできる

```
demo = gr.Interface(
    fn=my_func,
    inputs=[
      gr.Textbox(
        lines=2,  # 行数
        placeholder="Name Here..."  # プレースホルダ
      ),
      "checkbox", gr.Slider(0, 100)
    ]
    outputs=["text", "number"],
)

```


- 最後にWebUIを起動する  
シンプルに、  
```
demo.launch()
```
とするだけでよい

inputs には「クリアボタン」と「送信」ボタンが、outputs には「フラグする」ボタンが自動で追加される(フラグボタンは出力をローカルファイルに保存する)

簡易的なWebサーバが起動し、ブラウザ上でWeb UIが表示される

<img src="http://class.west.sd.keio.ac.jp/dataai/text/gradio1.jpg" width=700>


## より実践的な例

他にも様々な機能があるため、調べてみるとよい

```
import gradio as gr

# コールバック関数の定義
def callback_func(val1,val2,val3):
    return str(int(val1) * int(val2)), f"気温は {val3} 度です"

# 画面レイアウトの定義(Interfaceを使用)
app = gr.Interface(
    title="計算機",
    fn=callback_func,
    inputs=[
        gr.Textbox(label="入力欄1",lines=3, placeholder="ここに数値を入れてください..."),
        gr.Textbox(label="入力欄2",lines=5, placeholder="ここに数値を入れてください..."),
        gr.Slider(label="温度",minimum=0,maximum=100,step=1)
    ],
    outputs=[
        gr.Label(label="計算結果1",lines=3),
        gr.Textbox(label="計算結果2",lines=3)
    ]
    )

# Web UIの起動
app.launch(inbrowser=True)
```




### Blocks

簡易的に使用する場合はInterfaceを使い、より複雑なレイアウトを作る場合はBlocksを使う

- Interfaceのメリット

  - ショートカット文字列の使用が可能(Blocksでは使用不可)
  - Interfaceでは基本的なボタンを自動生成

- Blocksのメリット

  - Blocksでのみ利用可能なレイアウトがある
  - レイアウトが複雑になった場合でもwith文で可読性の高いコードが書ける

Blocksではレイアウトを指定でき、次のようなレイアウトがある



#### コンポーネントを横や縦に並べる  
gr.Row()やgr.Column()を利用する

コード中にコメントがあるので、切り替えてみるとよい

In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app:
  with gr.Row():
#  with gr.Column(scale=2):
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    outputs = gr.Textbox(label="挨拶")
    btn = gr.Button("クリックしてね!")
    # イベントハンドラー
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

#### タブで表示する  


In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app: # 入力タブを定義
  with gr.Tab("入力タブ"):
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    btn = gr.Button("クリックしてね!") # 出力タブを定義
  with gr.Tab("出力タブ"):
    outputs = gr.Textbox(label="挨拶")
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

#### 丸角の子要素を設ける

角が丸く、周囲にパディングがあるボックスである
- 以前はgr.Boxであったが、gr.Blocksに変更となった

In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app: # Box()関数でレイアウトを定義
  with gr.Group():
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    outputs = gr.Textbox(label="挨拶")
    btn = gr.Button("クリックしてね!") # イベントを定義
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

#### 子要素を折りたたみ可能にする

Accordionは、子要素を折りたたみ可能なセクションに配置する

openパラメータにより初期状態で開いているか(True)、閉じているか(False)を指定でき、デフォルトはOpen(True)


In [ ]:
import gradio as gr
def greet(name): return "Hello " + name + "!"
with gr.Blocks() as app: # Accordion()関数でレイアウトを定義
  with gr.Accordion(label="アプリを見る", open=False):
    inputs = gr.Textbox(placeholder="名前を入力してね!", label="名前")
    outputs = gr.Textbox(label="挨拶")
    btn = gr.Button("クリックしてね!") # イベントを定義
    btn.click(fn=greet, inputs=inputs, outputs=outputs)
app.launch()

### チャットを実装する

なんでもなく、"How are you?", "I know you", "I'm very hungry"のどれかをランダムに答えるアプリである

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["How are you?", "I know you", "I'm very hungry"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

(2023/12) 異常終了するため、ここで一度実行をとめる

In [ ]:
from google.colab import runtime
runtime.unassign()

# LLMチャットの実装



In [ ]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

In [ ]:
!pip uninstall -y --quiet tensorflow-probability

In [ ]:
!pip install --quiet openai cohere tiktoken

In [ ]:
!pip install --quiet chromadb kaleido python-multipart

In [ ]:
!pip install --quiet langchain-experimental

In [ ]:
!pip -q install gradio==3.48.0

In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

def chat(message: str) -> str:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    return llm.predict(message)


まずは、フレームワークとして、似非チャットを実装する
- ランダムに答えを返す

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

次に、過去の会話履歴を踏まえて回答するように chat関数を次のように更新する


In [ ]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage

langchain.verbose = True

def chat(message: str, history: ChatMessageHistory) -> str:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    messages = history.messages
    messages.append(HumanMessage(content=message))

    return llm(messages).content

実際に試してみよう

- 私の名前は出田です。
- 私の名前がわかりますか？

と入力すると、
「はい、先ほど出田さんとおっしゃいましたよね。」
といった回答になる

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()

# PDFドキュメントの内容をLangChainを用いて問い合わせる

ChatGPTには一度に扱えるテキストの量に限界があり、一度に入力させることはできない

そこで、巨大PDFの内容からテキストを抽出し、分割、テキスト間の関連性もつベクトルデータをベクターストアに格納する




In [ ]:
!pip install --quiet openai chromadb langchain pypdf tiktoken

必要となるライブラリを読み込む

In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader

API-KEYを読み込む
- エラーになる場合は、このテキストの最初にあるAPIキー設定箇所を再実行すること

In [ ]:
from dotenv import load_dotenv
load_dotenv(verbose=True)

テスト用PDFとして、米国CLOUD法のホワイトペーパーを利用する

In [ ]:
if not os.path.exists('wp.pdf'):
  !wget https://www.justice.gov/criminal-oia/page/file/1153436/download -O wp.pdf

PDFローダでテキスト化して読み込み、読分割する

In [ ]:
loader = PyPDFLoader("wp.pdf")
pages = loader.load_and_split()

5ページ目の文章を確認してみよう

In [ ]:
pages[5].page_content

ChatGPTを準備する

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

PDF文章をembeddingしてベクターストアに登録する

このベクターストアを LLM に与えることで、テキスト間の関連性について表現したベクトルデータを外部から与え、LLMで利用できるようになる

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

PDF ドキュメントへ自然言語で問い合わせる

ここでは、回答文の作成に関連した元テキスト群についても示すように指定する

In [ ]:
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

英語で US クラウド法とは何か？について問い合わせてみよう
- 回答は日本語を用いるように指示する

In [ ]:
query = "What is US Cloud Act? Answer within 50 words in Japanese."
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

result["answer"]

上記の回答に関連した元のテキスト群について確認する

In [ ]:
result['source_documents']

US クラウド法によってどういう主体が影響を受けるか問い合わせる

In [ ]:
chat_history = [(query, result["answer"]), (query, result["answer"])]
query2 = "CLOUD法によって影響を受ける主体にはどういったものがありますか？日本語で回答してください。"

result2 = pdf_qa({"question": query2, "chat_history": chat_history})

result2["answer"]

ChatGPT による回答の正確性については保証されていないが、難解な資料を素早く理解するにはかなり有効であるといえよう

# RAG (Retrieval Augmented Generative)

(A Cheat Sheet and Some Recipes For Building Advanced RAGで紹介されている内容)


## RAG の基本

RAGは、ユーザーの質問に対して、外部データベースから関連するドキュメントを取得し、そのドキュメントと元々のユーザーの質問をセットにしてLLMに渡され、LLMは、この内容をもとに回答を生成する手法

既に述べたLangChainのDictionaryがこれに相当する
- 目的はハルシネーションを抑え、最新の情報や、極めて専門的もしくは独自の情報を用いて回答を作成したいときなど

RAGにより以下が可能となる
- ハルシネーションやノイズを削減させる
- 情報不十分のとき回答しないようにする
- 複数の情報を参照して回答する
- 情報ソースの誤りを指摘できる

RAG品質は、次の指標で計測する
- コンテキストの関連性
- 回答の関連性
- 忠実度合



## RAGの応用

複雑な質問や高度なデータソースに対応するため、次の2つの重要な要件を満たすために高度な技術や戦略について述べる

- Retrieval（検索）: ユーザーの質問に最も関連性の高いドキュメントを見つけること
- Generation（生成）: 検索されたドキュメントを効果的に利用し、回答を生成すること

これを実現する技術的手法は以下の通りである

### 検索の要件に対する技術

- Chunk-size Optimization（チャンクサイズの最適化）:  
検索対象のドキュメントを適切なサイズに分割することで、情報の関連性を高める手法  
検索対象が多すぎる場合、効率が低下するため、適切なサイズのチャンクに分割することが重要

- Sliding Window Chunking（スライディングウィンドウチャンク分割）:  
情報の見落としを防ぐために、ドキュメントをオーバーラップしながらスライドさせて分割する手法  
重要な情報がチャンクの境界に分割されるのを防ぐ

- Structured Knowledge Integration（構造化ナレッジの統合）:   
構造化データ（例：データベースやスプレッドシート）を検索システムに統合し、より精度の高い検索結果を提供する手法

- Metadata Attachments（メタデータの付与）:   
メタデータ（例：著者、出版日、トピックなど）を利用して、検索精度を向上させる手法  
文脈に沿った関連性の高いドキュメントを絞り込むことができる

- Knowledge Graphs（ナレッジグラフ）:   
ナレッジグラフを活用し、ドキュメント間の関連性を明確にする手法  
情報同士の関係性を視覚化することで、検索の精度が向上する

- Mixed Retrieval（混合検索）:   
複数の検索アルゴリズムを組み合わせ、情報源から最も有用なドキュメントを取得する手法  
異なるアプローチを組み合わせることで、異なる角度から関連性が高められる

- Question Embedding Transformation（質問の埋め込み変換）:  
 質問を埋め込み表現に変換し、情報と質問の一致を効果的に計算する手法  
質問の文脈を理解し、より関連性の高い検索結果が取得できる

### 生成の要件に対する技術

- Information Compression（情報の圧縮）:   
生成モデルが扱いやすい形に情報を圧縮する手法  
例えば、ノイズを減らしたり、重要な情報を抽出してから生成を行う

- Generator Fine-Tuning（生成モデルの微調整）: 検索されたドキュメントに基づいて、生成モデルを特定のタスクに適応させるための微調整を行う手法  
生成された回答の精度が向上する

- Result Re-Ranking（結果の再ランキング）: 一度生成された結果を再度評価し、最も関連性が高いものを再ランキングする手法  
ユーザーに提供する最終結果の品質が向上する

- Adapter Methods（アダプターメソッド）: 生成モデルに外部情報を柔軟に取り入れるための手法  
新しい情報を簡単に生成モデルに適応させることができる

### 同時に2つの要件を満たす技術
- Monolithic Fine-Tuning（単一モデルの微調整）:   
検索と生成の両方を同時に最適化するために、単一の大規模モデルを微調整する手法  
情報検索と生成のプロセスが統合され、シームレスな処理が可能になる

- Retrieval-Foundational Model（検索基盤モデル）:   
検索モデルそのものを基礎モデルに統合し、検索結果と生成がより密接に結びついた形で動作するように設計する手法

- Generator-Enhanced Retrieval（生成強化型検索）:   
生成モデルを利用して、検索結果のリランキングやフィルタリングを行う手法  
生成モデルが得た情報をさらに効果的に検索に活用できる

- Iterative Retrieval-Generation（反復検索生成）:   
検索と生成のプロセスを繰り返し行い、精度の高い回答を得る手法  
例えば、一度検索された結果を元に、再度質問を修正しながらプロセスを繰り返す

### RouterRetriever

RAGで複数の埋め込みモデルを利用し、文書検索精度を向上させる手法

(KAISTとアレン人工知能研究所により提案)

通常のRAGは、埋め込みモデル（エンべディングモデル）を1つだけ利用して、ベクトル検索する
- ユーザーの質問とソースとなる大量の文書を、ただ一つの同じ埋め込みモデルでベクトル化して、検索している
- この埋め込みモデルとして、OpenAIが公開している「text-embedding」といった汎用的モデルの利用が一般的

汎用的な埋め込みモデルではなく、そのドメインに特化したモデルを利用することで検索精度が高くなる

RouterRetrieverは、ユーザーの質問のジャンルに応じて、複数の埋め込みモデルを使い分ける手法

#### 従来手法における問題点

従来はユーザーの質問に専門的な内容が入っている場合、文書検索が適切に行えなかった

これは、RAGの文書検索がもつ以下のジレンマが関係している

- RAGである分野の検索に強くするには、その分野に特化した埋め込みモデルにするのが一番よい
- 一方でその分野に特化すると、それ以外の質問には、弱くなる

現状は、これに対し、全ての分野に平均的に強い「バランス型」の埋め込みモデルを1つだけ使うという方針であった
- どのような問いあわせでも平均的な性能が獲得できる

性能を上げるには、ハイブリッド学習のように、複数の埋め込みモデルを動的に使い分ける
- ユーザーからの質問に対して、「どの埋め込みモデルを使うの最適か」を判断し、そのモデルで検索を行う


#### 事前にやっておくこと

- 専門家モデル（例えば、医学特化の埋め込みモデルなど）を複数作成しておく
- `Contriever`という汎用的な埋め込みモデルをLoRA（Low-Rank Adaptation）して複数作成しておく
  - ユーザーの質問には複数のモデルを使い分けますが、ドキュメントの埋め込みには、通常通り1つの埋め込みモデル（Contriever）のみを利用
- 「どの埋め込みモデルを使うの最適か」を、動的に判断するための仕組みを構築
- ユーザーが質問を入力してきた際に、ユーザーの質問を汎用的なモデル（Contriever）でベクトル化
  - このベクトルと、事前に用意した「各専門家モデルが得意なタイプの質問のベクトル」を比較

#### 埋め込みモデルの選択

で最もスコアが高かったモデルを利用して、もう一度ユーザーの質問をベクトル化

このベクトルを利用して、検索対象である、大量の文書を検索（ここは通常のRAGと同じ）


BEIRベンチマークにおいて、MSMARCOで訓練された単一モデルに比べて+2.1、マルチタスク訓練モデルに比べて+3.2のnDCG@10スコア向上を達成
ゲート（専門家モデル）の数を増やすにつれて、一貫して性能が向上
ドメインに特化した専門家モデルがないデータセットに対しても汎用性を持つことが確認されている

# その他トピック

ここ3ヵ月で次の通り、追いかけるのも大変
- 1年たち、既に古すぎる情報となった

## DALL-E3
画像生成AIの中でも高精度・高品質画像を作成可能なAI

## SDXL Turbo
リアルタイムでプロンプトに反応して画像生成
https://clipdrop.co/ja/stable-diffusion-turbo

## Adobe Firefly
ようやくAdobeも動きだし、著作権・商用利用問題クリアの画像生成AIを公開

## Notion AIやCursorなどのLLM統合
- Notionは様々な機能を持つメモ帳
  - AI拡張は有料
- Cursorはプログラミング用統合環境
  - VSCodeオワコンといわれている

## Claude
- ChatGPT並み(かそれ以上)の精度を持つLLM

## Google Gemini
- Googleとの連携がとりやすく、使い勝手の良いLLM
- Colabとも連携

## Runway Gen-2
画像から高精細かつ高品質な動画を生成
- 動かしたいものを指定したり、プロンプトで指示することができる


# 課題1(LangChain)

上記のPDFファイル解析アプリケーションの例を、LangChainを用いたチャットボット形式に修正しなさい

# 課題2(LangChainによるチューニング)

履歴付きチャットボットを改造し、常に回答が子供の対応であるようにプロンプトエンジニアリングを行いなさい

単純に、入力問い合わせに対して、「考え方や言葉遣いを6歳の子供のようにして答えなさい」といった言葉を付け加えなさい

# 課題3(Embedchain)

Embedchainを用いて、何かしら専門的な内容に対して回答可能とするチャットボットを実装しなさい

- ChatGPTへの質問結果も示し、実際に回答が改善されていることを示しなさい。
- インプットトークンが足りないというエラーが出た場合は、次の方法によりGPT4への切り替えなさい
  - gpt4を指定したconfigファイルを作成・配置する
  - [推奨] App()で初期化する際に `App.from_config(config_path="config_file.yaml")`とする

# 課題4(GPTs)

ChatGPTのGPTsを試しなさい

この課題を解くには、ChatGPT4を利用する必要があるため、課金が伴う点に注意しなさい

例えば、

- 政府発行の文章や、博物館などの解説ページなどの情報を参照し、専門的な内容に回答するチャットを作成する
  - 会社の内部文章などが面白いが、普通に規定でできないであろう
  - 仮想的にそのような「公開されていない情報」(自分の日記など)について試すとよい
- 日本語で必ず回答するようにする
- 発言の最後に、専門用語についての解説を付与するようにする

など

また、GPTsにおけるAPI操作機能を用いて、気温や湿度などを住所から入手し、適切な服装などを指示するGPTを作成しなさい

https://weather.tsukumijima.net/ などを利用するとよい

さらに、HotPepper APIを用いて、お店を検索するGPTを作成しなさい
- これについて、HotPepper側の制限によりアクセスが拒否されている場合がある
- この場合は、Proxyを立てること